# Banking and Unemployment
---
The below script explores the relationship between states with high unemployment rates and bank counts per state.

In this script, we retrieved and plotted data from the 2013 US Census and Google Places API to show the relationship between various socioeconomic parameters and bank count across 700 randomly selected zip codes. We used Pandas, Numpy, Matplotlib, Requests, Census API, and Google API to accomplish our task.

In [13]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
import us


# Census API Key
c = Census(census_key, year=2016)

In [19]:
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})

[{'NAME': 'Maryland', 'B25034_010E': 133511.0, 'state': '24'}]

## Data Retrieval

In [44]:
# Run Census Search to retrieve data on all states (2016 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(('NAME'          #state name
                          ,'B01003_001E'  #population total
                          ,'B17001_001E'  #poverty status total
                          ,'B17001_002E'  #poverty status below poverty level
                          ,'B19301_001E'  #total per capita income
                          ,'B23025_001E'  #total labor force 16 years and over
                          ,'B23025_004E'  #total Civilian force 16 years and over, employed
                          ,'B23025_005E'  #Unemployment Civilian Count
                          ,'B23025_006E'  #total Armed forces 16 years and over, employed
                          ,'B23025_007E'  #total not labor force
                          ,'B25003_001E'  #total houses
                          ,'B25003_002E'  #Total!!Owner occupied
                          ,'B25003_003E'  #Total!!Renter occupied
                          ,'B25081_002E'  #Houses with mortgage
                          ,'B992701_001E' #total people
                          ,'B992701_002E' #people with medical coverage
                          ,'B992701_003E' #people without medical coverage
                          ), {
                         'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# # # Column Reordering
census_pd = census_pd.rename(columns={ 'NAME': 'State'        #state name
                                      ,'B01003_001E':'Total Population'  #population total
                                      ,'B17001_001E':'Total Poverty'  #poverty status total
                                      ,'B17001_002E':'People Below Poverty Level'  #poverty status below poverty level
                                      ,'B19301_001E':'Per Capita Income'  #total per capita income
                                      ,'B23025_001E':'Total Labor Force'  #total labor force 16 years and over
                                      ,'B23025_004E':'Total Employed Civilian'  #total Civilian force 16 years and over, employed
                                      ,'B23025_005E':'Unemployment Civilian'  #Unemployment Civilian Count
                                      ,'B23025_006E':'Armed Forces Employment'  #total Armed forces 16 years and over, employed
                                      ,'B23025_007E':'Total Not Labor Force'  #total not labor force
                                      ,'B25003_001E':'Total Houses'  #total houses
                                      ,'B25003_002E':'Houses with owners'  #Total!!Owner occupied
                                      ,'B25003_003E':'Houses with renters'  #Total!!Renter occupied
                                      ,'B25081_002E':'Houses with mortgage'  #Houses with mortgage
                                      ,'B992701_001E':'Total People' #total people
                                      ,'B992701_002E':'Total People with insurance' #people with medical coverage
                                      ,'B992701_003E':'Total People withou insurance'}) #people without medical coverage

# # Add in Employment Rate (Employment Count / Population)
# census_pd["Unemployment Rate"] = 100 * \
#     census_pd["Unemployment Count"].astype(
#         int) / census_pd["Population"].astype(int)

# # Final DataFrame
# census_pd = census_pd[["Zipcode", "Population", "Unemployment Rate"]]



52


,Total Population,Total Poverty,People Below Poverty Level,Per Capita Income,Total Labor Force,Total Employed Civilian,Unemployment Civilian,Armed Forces Employment,Total Not Labor Force,Total Houses,Houses with owners,Houses with renters,Houses with mortgage,Total People,Total People with insurance,Total People withou insurance,State,state
0,4841164.0,4720592.0,868666.0,24736.0,3864302.0,2042025.0,184479.0,12150.0,1625648.0,1851061.0,1267824.0,583237.0,730637.0,4761291.0,657349.0,4103942.0,Alabama,01
1,736855.0,719064.0,72826.0,34191.0,568732.0,353954.0,30139.0,16382.0,168257.0,250235.0,159313.0,90922.0,103704.0,712964.0,83618.0,629346.0,Alaska,02
2,6728577.0,6573523.0,1165636.0,26686.0,5290839.0,2879372.0,249972.0,17373.0,2144122.0,2448919.0,1531878.0,917041.0,991361.0,6620233.0,1024109.0,5596124.0,Arizona,04
3,2968472.0,2881404.0,542431.0,23401.0,2339307.0,1266552.0,93190.0,4445.0,975120.0,1141480.0,749452.0,392028.0,417765.0,2915402.0,346839.0,2568563.0,Arkansas,05
4,38654206.0,37913144.0,6004257.0,31458.0,30565746.0,17577142.0,1683726.0,130452.0,11174426.0,12807387.0,6929007.0,5878380.0,5000119.0,38151997.0,4892528.0,33259469.0,California,06


In [45]:
# Visualize
print(len(census_pd))
census_pd

52


,Total Population,Total Poverty,People Below Poverty Level,Per Capita Income,Total Labor Force,Total Employed Civilian,Unemployment Civilian,Armed Forces Employment,Total Not Labor Force,Total Houses,Houses with owners,Houses with renters,Houses with mortgage,Total People,Total People with insurance,Total People withou insurance,State,state
0,4841164.0,4720592.0,868666.0,24736.0,3864302.0,2042025.0,184479.0,12150.0,1625648.0,1851061.0,1267824.0,583237.0,730637.0,4761291.0,657349.0,4103942.0,Alabama,01
1,736855.0,719064.0,72826.0,34191.0,568732.0,353954.0,30139.0,16382.0,168257.0,250235.0,159313.0,90922.0,103704.0,712964.0,83618.0,629346.0,Alaska,02
2,6728577.0,6573523.0,1165636.0,26686.0,5290839.0,2879372.0,249972.0,17373.0,2144122.0,2448919.0,1531878.0,917041.0,991361.0,6620233.0,1024109.0,5596124.0,Arizona,04
3,2968472.0,2881404.0,542431.0,23401.0,2339307.0,1266552.0,93190.0,4445.0,975120.0,1141480.0,749452.0,392028.0,417765.0,2915402.0,346839.0,2568563.0,Arkansas,05
4,38654206.0,37913144.0,6004257.0,31458.0,30565746.0,17577142.0,1683726.0,130452.0,11174426.0,12807387.0,6929007.0,5878380.0,5000119.0,38151997.0,4892528.0,33259469.0,California,06
5,5359295.0,5241057.0,637938.0,33230.0,4245559.0,2692529.0,171695.0,33135.0,1348200.0,2051616.0,1320617.0,730999.0,959882.0,5267878.0,716726.0,4551152.0,Colorado,08
6,3588570.0,3479672.0,360464.0,39906.0,2911925.0,1793688.0,155005.0,8367.0,954865.0,1354713.0,900223.0,454490.0,625486.0,3535139.0,462556.0,3072583.0,Connecticut,09
7,934695.0,908973.0,109448.0,31118.0,753039.0,438292.0,33669.0,2860.0,278218.0,348051.0,246940.0,101111.0,166435.0,920268.0,129412.0,790856.0,Delaware,10
8,659009.0,624954.0,112060.0,48781.0,554780.0,348225.0,33326.0,3505.0,169724.0,276546.0,112672.0,163874.0,86201.0,648263.0,100874.0,547389.0,District of Columbia,11
9,19934451.0,19514334.0,3139258.0,27598.0,16339299.0,8755427.0,802016.0,56151.0,6725705.0,7393262.0,4787320.0,2605942.0,2808279.0,19621207.0,2976442.0,16644765.0,Florida,12


## Combine Data

In [15]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
census_data_original = pd.read_csv(
    "../Resources/zip_bank_data.csv", dtype="object", encoding="utf-8")

# Visualize
census_data_original.head()

,Zipcode,Address,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777,22137,13.18375282,42.6306916,-85.2929384,9
1,71405,"Ball, LA 71405, USA",5687,35.8,55242,23941,15.73764727,31.4061799,-92.396174,12
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944,35961,9.295946358,38.2201614,-76.4967919,9
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125,18884,24.40944882,35.7310368,-87.2419299,0
4,58466,"Marion, ND 58466, USA",509,44.5,69844,36981,9.430255403,46.5594224,-98.3481542,1


In [16]:
# Merge the two data sets along zip code
census_data_complete = pd.merge(
    census_data_original, census_pd, how="left", on=["Zipcode", "Zipcode"])

# Save the revised Data Frame as a csv
census_data_complete.to_csv(
    "../Resources/bank_data_with_employment.csv", encoding="utf-8", index=False)

# Visualize
census_data_complete.head()

,Zipcode,Address,Population_x,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count,Population_y,Unemployment Rate
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777,22137,13.18375282,42.6306916,-85.2929384,9,19031.0,5.312385
1,71405,"Ball, LA 71405, USA",5687,35.8,55242,23941,15.73764727,31.4061799,-92.396174,12,5687.0,4.571830
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944,35961,9.295946358,38.2201614,-76.4967919,9,6562.0,4.053642
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125,18884,24.40944882,35.7310368,-87.2419299,0,762.0,10.236220
4,58466,"Marion, ND 58466, USA",509,44.5,69844,36981,9.430255403,46.5594224,-98.3481542,1,509.0,1.178782


## Heatmap of poverty rate

In [18]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [19]:
# Store 'Lat' and 'Lng' into  locations 
locations = census_data_complete[["Lat", "Lng"]].astype(float)
print(locations)
# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
poverty_rate = census_data_complete["Poverty Rate"].astype(float)
print(poverty_rate)

           Lat         Lng
0    42.630692  -85.292938
1    31.406180  -92.396174
2    38.220161  -76.496792
3    35.731037  -87.241930
4    46.559422  -98.348154
5    39.564892 -115.994099
6    38.979335  -96.004412
7    36.915294 -120.205110
8    34.485433 -117.420397
9    42.446414  -85.581791
10   42.990239  -88.498499
11   42.662874  -78.690926
12   31.269297  -92.046915
13   42.010529  -87.692626
14   39.900841  -85.961108
15   34.335049 -116.537392
16   41.312980  -86.032563
17   47.079517 -116.384577
18   40.907107  -73.821321
19   40.072034  -94.665703
20   40.348083  -81.438886
21   32.037114  -81.174750
22   36.333802  -81.130639
23   38.497733  -90.843688
24   40.036199  -92.947566
25   40.638523  -74.015323
26   32.075509  -87.766493
27   36.211921  -81.526728
28   42.874922  -88.614484
29   33.974291 -118.352039
..         ...         ...
670  46.953982 -113.912010
671  30.225199  -93.359331
672  38.631723  -77.252010
673  41.643113  -74.448166
674  39.697014  -76.722108
6

In [20]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
#heat_layer.dissipating = False
#heat_layer.max_intensity = 100
#heat_layer.point_radius = 1

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

![Heatmap](../Images/heatmap.png)

In [21]:
# Convert bank rate to list
bank_rate = census_data_complete["Bank Count"].tolist()

In [22]:
# Create bank symbol layer
bank_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)


fig = gmaps.figure()
fig.add_layer(bank_layer)

fig

Figure(layout=FigureLayout(height='420px'))

![Bank Map](../Images/bank_map.png)

In [23]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(bank_layer)

fig

Figure(layout=FigureLayout(height='420px'))

![Combined Map](../Images/final_map.png)